# VLF data processor
## Vojtěch Laitl 2016
### Ionozor Measuring Network - VLF monitors data anylysis and graph processing

This is an Juypter notebook made for SID monitors and interactive viewing of the processed VLF data. The computations are covered by a simplified ionospheric plasma physics model and based on Python modules. The script does not save any pictures, nor data given. The supporting files are deleted when the calculations and plotting are finished, so the notebook works only with the primary data measured.

# Prerequisities

    sudo apt-get install python3-matplotlib python3-numpy python3-scipy python3-astropy


In [5]:
import bzpost
import datetime, requests, os, glob
import time
import numpy as np
import scipy
import astropy.io 
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import download_file
import matplotlib.pyplot as plt
from scipy.integrate import quad
import cmath
import scipy.interpolate as interpol
from mpl_toolkits.mplot3d import Axes3D
import os
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

First of all, the libraries needed for the script are defined and imported. The notebook works with the bzpost module (https://github.com/bolidozor/python-bolidozor-postprocessing/tree/master) installed.

In [21]:
output_slider_variable = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False)
def f(station):
    output_slider_variable.value = str(station)

interact(f,station=['svakov/R2','OBSUPICE/R2']);

interactive(children=(Dropdown(description='station', options=('svakov/R2', 'OBSUPICE/R2'), value='svakov/R2'), Output()), _dom_classes=('widget-interact',))

After the important modules are imported, you are able to set the observatory from where the data will be collected.

In [22]:
if output_slider_variable.value == 'OBSUPICE/R2':
    sourceUrl = 'http://space.astro.cz/ionozor/VLF/OBSUPICE/OBSUPICE_VLF_R2/'
if output_slider_variable.value == 'svakov/R2':
    sourceUrl = 'http://space.astro.cz/ionozor/VLF/svakov/SVAKOV_VLF_R2/'
print('Your observatory and source URL are')
print(output_slider_variable.value)
print(sourceUrl)

Your observatory and source URL are
svakov/R2
http://space.astro.cz/ionozor/VLF/svakov/SVAKOV_VLF_R2/


Next, your choice is converted into Python friendly format. If successfully done, you will be informed by an output printed.

In [25]:
output_slider_variable1 = widgets.Text()
def f(year):
    output_slider_variable1.value = str(year)    
interact(f,year='2017')
output_slider_variable2 = widgets.Text()
def f(month):
    output_slider_variable2.value = str(month)    
interact(f,month='04')
output_slider_variable3 = widgets.Text()
def f(day):
    output_slider_variable3.value = str(day)    
interact(f,day='02')
output_slider_variable4 = widgets.Text()
def f(hour):
    output_slider_variable4.value = str(hour)    
interact(f,hour='18')
output_slider_variable5 = widgets.Text()
def f(minute):
    output_slider_variable5.value = str(minute)    
interact(f,minute='30')
print('Let us start! Please set your request for the initial time to download your .fits file.')

Let us start! Please set your request for the initial time to download your .fits file.


After that, let's choice the initial time of your data collection just by typing your requests to the cells appeared. Don't worry about apostrophes or any other special characters, just insert the plain text but, please, respect the datetime format YYYY-MM-DD-HH-mm for the year, month, day, hours and minutes.

In [26]:
year_start = np.asarray(output_slider_variable1.value,dtype='int')
month_start = np.asarray(output_slider_variable2.value,dtype='int')
day_start = np.asarray(output_slider_variable3.value,dtype='int')
hour_start = np.asarray(output_slider_variable4.value,dtype='int')
minute_start = np.asarray(output_slider_variable5.value,dtype='int')
print('The initial time conditions are given by the array')
print(year_start,month_start,day_start,hour_start,minute_start)

The initial time conditions are given by the array
2017 4 2 18 30


Then, the parameters are converted into the iPython-friendly format and, as you've got used to, you will be informed by a printed output.

In [30]:
output_slider_variable6 = widgets.Text()
def f(year):
    output_slider_variable6.value = str(year)    
interact(f,year='2017')
output_slider_variable7 = widgets.Text()
def f(month):
    output_slider_variable7.value = str(month)    
interact(f,month='04')
output_slider_variable8 = widgets.Text()
def f(day):
    output_slider_variable8.value = str(day)    
interact(f,day='02')
output_slider_variable9 = widgets.Text()
def f(hour):
    output_slider_variable9.value = str(hour)    
interact(f,hour='18')
print('Let us start! Please set your request for the initial time to download your .fits file.')
output_slider_variable10 = widgets.Text()
def f(minute):
    output_slider_variable10.value = str(minute)    
interact(f,minute='30')
print('Thank you, let us carry on! Please set your request for the terminal time to download your .fits file.')

Thank you, let us carry on! Please set your request for the terminal time to download your .fits file.


In [31]:
year_end = np.asarray(output_slider_variable6.value,dtype='int')
month_end = np.asarray(output_slider_variable7.value,dtype='int')
day_end = np.asarray(output_slider_variable8.value,dtype='int')
hour_end = np.asarray(output_slider_variable9.value,dtype='int')
minute_end = np.asarray(output_slider_variable10.value,dtype='int')
print('The terminal time conditions are given by the array')
print(year_end,month_end,day_end,hour_end,minute_end)

The terminal time conditions are given by the array
2017 4 2 18 30


In the two cells above, the same functions are run for the terminal time conditions. Please, work with the interactive input by the same way like for the initial time.

In [32]:
def download_file(url):
    local_filename = url.split('/')
    # note the stream=True parameter
    r = requests.get(url, stream=True)
    with open('local_filename', 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
    return local_filename

download_file(sourceUrl)

['http:',
 '',
 'space.astro.cz',
 'ionozor',
 'VLF',
 'svakov',
 'SVAKOV_VLF_R2',
 '']

Next, the data downloading is solved, called by a print output and set up to be run. The source URL is defined above within the datetime parameters and being set for each observatory and station.

In [33]:
def download_fits(dir, year_start, month_start, day_start, hour_start, minute_start, year_end, month_end, day_end, hour_end, minute_end):
    '''
    Creates subdirectory 'snapshots' and downloads set of fits images from http://space.astro.cz/bolidozor/ from time period given in arguments. Returns nothing.
    >>>import getboli
    >>>getboli.download_fits('snapshots',2015,8,8,4,1,2015,8,9,6,1)
    >>>
    '''
    collector = []
    wd = os.getcwd()
    if not os.path.exists(dir):
        os.makedirs(dir)
    os.chdir(dir)
    con = bzpost.HTTPConnector(sourceUrl)
    con.connect()

    for snapshot in con.get_snapshots(datetime.datetime(year_start, month_start, day_start, hour_start, minute_start), datetime.datetime(year_end, month_end, day_end, hour_end, minute_end)):
       collector.append(snapshot.url) 

    for url in collector:
        download_file(url)

    con.close()
    os.chdir(wd)
	
download_fits('snapshots', year_start, month_start, day_start, hour_start, minute_start, year_end, month_end, day_end, hour_end, minute_end)
print('The directory and datetime are given by:')
print('snapshots', year_start, month_start, day_start, hour_start, minute_start, year_end, month_end, day_end, hour_end, minute_end)

AttributeError: 'function' object has no attribute 'urlparse'

Now, the datetime functions are set into a tool which covers the parameters of data downloading.

In [ ]:
def mkmosaic(dir, output='out.fits', axis='y', part=1, showplot=True):
    '''
    Concaternate specified parts of images from directory along the geometric axis. Plots the output by default. Having something else than fits files in target directory will likely result in error.
    :dir: The directory in which fits images are situated.
    :output: The filename of the output concaternated image.
    :axis: The axis in cartesian coordinate system along which the pictures are added.
    :part: What part of the fits images should be added. Default fits images have header on 0 and image part on 1. Raw files have each channel on 0, 1, 2 without a header.
    :showplot: If set to True the output is drawn using matplotlib.
    :return: Nothing.
    >>>import getboli
    >>>getboli.mkmosaic('snapshots', 'out.fits', axis='x', part='0')
    >>>
    TODO: .*\.fits regex
    '''
    for image in os.listdir(dir):
        hdulist = fits.open(os.path.join(dir, image))
        if 'a' in locals():
            if axis == 'x':
                a = np.append(a, hdulist[part].data, axis=1)
            if axis == 'y':
                a = np.append(a, hdulist[part].data, axis=0)
            if axis == 'z':
                a = np.add(a, hdulist[part].data)
        else:
            a = hdulist[part].data #matrix representing the image part
        hdulist.close()
    hdu = fits.PrimaryHDU(a)
    hdu.writeto(output)
    if showplot:
        plt.imshow(a)
        plt.show()
    del a #when axis were switched the variable caused skip of the else condition

mkmosaic('snapshots', 'out.fits', 'y', 1, False)

Definitely, the data are downloaded from the station's website and saved into the file of out.fits. Because of the next steps, no plot is drawn right now.

In [ ]:
data_list = fits.open('out.fits')
data_list.info()
data = data_list[0].data
data_list.close

The data downloaded have just been written into a 2-D numpy array and are stored witihin the data chain since this step.

In [ ]:
epsilon = 8.8542e-12
e = 1.602e-19
k_B = 1.38e-23
gamma = 0.001
R_inf = 3.2899e15
c = 3e8
b = 2.898e-3
h = 6.626e-34
m_el = 9.109e-31
R = 8.314
N_A = 6.022e23
lambda_De = 3.10399215e-05
I0 = -26.74
R = 1.496e11
i = scipy.sqrt(-1)

In the cell above, all of the constants used for the oncoming calculations are defined.

In [ ]:
nu = 18000+(16+2/3)*np.arange(0,len(data))
t = np.arange(0,len(data[0]))*3600/len(data[0])
L = -data*1000
n0 = -1/4*L**-4
n0[np.isinf(n0)]=0
E_k = -36*np.pi**-2*epsilon**-4/3*n0*e**4
T = -E_k/k_B
d_lambda_De = np.sqrt(np.abs((epsilon*k_B*T)/(n0*e**2)))
p = np.diff(E_k,0)
nu_delta = p*c/h
omega = (nu+nu_delta.reshape((len(nu_delta[0]),len(nu)))).reshape((len(nu_delta),len(nu_delta[0])))
n = (omega**2*m_el*epsilon)/e**2
T_el = np.abs(T*n)
T_el = T_el+190.05
T_el[T_el>=10**4]=0
n[n>9.99999*10**8]=0
n = (n.reshape((len(nu_delta[0]),len(nu)))+(nu**2*m_el*epsilon)/e**2).reshape((len(nu_delta),len(nu_delta[0])))
dn = np.diff(n,0)
N_D = 4/3*np.pi*lambda_De*n
N = (n.reshape((len(nu_delta[0]),len(nu)))-(nu**2*m_el*epsilon)/e**2).reshape((len(nu_delta),len(nu_delta[0])))
Bl = 1+(0.01*L)
mD = 10e-10
h0 = np.pi*Bl/(2+2*mD)
h0_t = (10*h0)**2/10
f_n = dn*Bl/(N*(1-mD))
f_n2 = f_n**2
f_ln = np.log(1/2*f_n-((f_n2**2 - 4)**1/2))
h_t = np.abs((Bl*f_ln + 2*np.pi*i)/(1-mD))
H = np.nan_to_num(10*(h0_t + h_t)/3)
H[H<59.999]=0
Is = np.exp(L/10)
fce = Is*R**2/I0
alpha = scipy.arccos(fce)
s = H*np.tan(np.abs(alpha))

By the time the data are downloaded and the mathematical constants are set, the equations involved in the simplified model are run. The cover the basic plasma parameters and the ionospheric drift characteristics.

In [ ]:
n_mean = np.mean(n,axis=1)
T_mean = np.mean(T_el,axis=1)
H_mean = np.mean(H,axis=1)*3
s_mean = np.mean(s,axis=1)*3
data_mean = np.mean(data,axis=1)
L_mean = np.mean(L,axis=1)
f = interpol.interp1d(nu,H_mean,kind='quadratic')
F = f(nu)
g = interpol.interp2d(nu,data_mean,L_mean)
G = g(nu,data_mean)

After that, the data output is simplified from the 2-D array into a single-axis chain containing the data used for the plotting.

In [ ]:
fig1 = plt.figure()
plt.imshow(data)
plt.colorbar()
plt.title('Spectrograph')
plt.xlabel('Time index')
plt.ylabel('Frequency index')
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection='3d')
ax.plot_surface(nu,data_mean,G)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('L (dB)')
ax.set_zlabel('dn/dL (1/m3)')
ax.set_title('Plasma discharges')
fig3 = plt.figure()
plt.plot(nu,H_mean,'go'),plt.plot(nu,F)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Height (km)')
plt.title('Ionogram')
fig4 = plt.figure()
plt.plot(n_mean,T_mean,'go')
plt.xlabel('Electron density (1/m3)')
plt.ylabel('Temperature (K)')
plt.title('Charged particles')
fig5 = plt.figure()
plt.plot(H_mean,s_mean,'go')
plt.xlabel('Vertical drift (km)')
plt.ylabel('Horizontal drift (km)')
plt.title('Ionospheric drift')

Finally, the simplified data output is drawn using simple figures.

In [ ]:
plt.show()

As the final result, the figures drawn are shown using matplotlib.
#### Spectrograph
    The spectrograph plot shows the measured data themselves. The radio signal intensity level is shown within a 2-D char where the x-axis shows the time index and the y-axis the frequency index of the relative intensity of radio waves monitored.
#### Plasma discharges
    Plasma discharges plot covers the inital step of the mathematical model. The frequency set and the relative intensity measured are compared with each other and interlanded with the basic equation of the plasma physics model 
\begin{equation}
\frac{\partial{n}}{\partial{L}} \approx - \frac{1}{4}L^{-4}
\end{equation}
#### Ionogram
    The next graph is the standard ionogram, a digram which shows the relation between the frequency (electron plasma frequency and thus mean electron density, respectively) and the height where the ionospheric plasma is being located. The ionogram involves both measured daa ad interpolaton.
#### Charged particles
    As the result of plasma paramters modelling, a simple diagram showing the electron density and plasma temperature is given.
#### Ionospheric drift
    The last step of this data processing tool is the diagram which shows the ionospheric plasma distribution involving both vertical distribution and horizontal distribution; they both are able to be reached within the ionogram.

In [ ]:
os.remove('out.fits')
os.remove('local_filename')

Having satisfied the data processing, the temporary supporting files containing the data measured are deleted. If you do want to save the data processed, please use our GitHub to download an iPython Jupyter notebook which would be more suitable for you.